In [1]:
import tensorflow as tf
import numpy as np
import time
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
np.set_printoptions(suppress=True)

Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [36]:
# Parameters
learning_rate = 0.001
training_epochs = 10
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)
print("total_batch:", total_batch)

# Network Parameters
n_input = 784
n_hidden_1 = 256
n_hidden_2 = 256
n_classes = 10

# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y_true = tf.placeholder(tf.float32, [None, n_classes])


# Create model
def nn(x, weights, biases):
    # Hidden layer 1 with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer 2 with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer


# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
y_pred = nn(x, weights, biases)
y_softmax = tf.nn.softmax(y_pred)

# Define loss and optimizer
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_pred, labels=y_true))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

pred = tf.argmax(y_pred, 1)
true = tf.argmax(y_true, 1)
correct_prediction = tf.equal(pred, true)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

total_batch: 550


In [37]:
# Training cycle
start = time.time()
for epoch_i in range(training_epochs):
    ave_cost = 0
    for batch_i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        _, c = sess.run(
            [optimizer, loss],
            feed_dict={
                x: batch_xs,
                y_true: batch_ys
            })
        ave_cost += c / total_batch
    # Display logs per epoch step
    if epoch_i % 1 == 0:
        train_acc = accuracy.eval(feed_dict={x: batch_xs, y_true: batch_ys})
        test_acc = accuracy.eval(feed_dict={
            x: mnist.test.images,
            y_true: mnist.test.labels
        })
        print("Epoch:%3d Batch:%5d " % (epoch_i,
                                        batch_i), "train_acc=%.5f" % train_acc,
              "test_acc=%.5f" % test_acc, "train_cost=%.9f" % ave_cost)
end = time.time()
print("Process Time :%.2f s" % (end - start))

# Calculate accuracy
print(
    sess.run(
        accuracy, feed_dict={
            x: mnist.test.images,
            y_true: mnist.test.labels
        }))

Epoch:  0 Batch:  549  train_acc=0.82000 test_acc=0.85680 train_cost=170.935488340
Epoch:  1 Batch:  549  train_acc=0.90000 test_acc=0.89120 train_cost=42.310539426
Epoch:  2 Batch:  549  train_acc=0.92000 test_acc=0.91020 train_cost=26.825713651
Epoch:  3 Batch:  549  train_acc=0.94000 test_acc=0.91610 train_cost=18.475384254
Epoch:  4 Batch:  549  train_acc=0.95000 test_acc=0.92360 train_cost=13.140156484
Epoch:  5 Batch:  549  train_acc=0.95000 test_acc=0.92770 train_cost=9.962927763
Epoch:  6 Batch:  549  train_acc=0.94000 test_acc=0.93280 train_cost=7.574798276
Epoch:  7 Batch:  549  train_acc=0.99000 test_acc=0.93680 train_cost=5.833056958
Epoch:  8 Batch:  549  train_acc=0.98000 test_acc=0.93500 train_cost=4.179387725
Epoch:  9 Batch:  549  train_acc=1.00000 test_acc=0.93640 train_cost=3.590055896
Process Time :48.12 s
0.9364
